# Deep learning - CocoDoom
## Introduction
Dans ce rapport nous allons vous présenter notre travail de deep learning sur le dataset CocoDoom. Ce dataset est composé d'images extraites de 3 parties de Doom, divisés chacunes en 32 maps. Les images sont des captures d'écran du jeu, et sont labellisées avec les objets présents dans l'image (bounding box et catégorie). Le but de ce projet est de créer un modèle capable de prédire les objets présents dans ces images. 

Nous avons décidé d'utiliser yolo pour ce projet car il permet de faire de la classification ainsi que de la segmentation d'images.

## Préparation des données
La première étape fût de préparer les données extraites du dataset, celles ci étant au format MS Coco ( Bounding box = [x_min, y_min, width, height]), Nous avons donc dû les convertir au format Yolo (Bounding box = [x_center, y_center, width, height]). Attention, au format yolo les données sont normalisées par rapport à la taille de l'image.


In [2]:
import shutil
import os 
import json
import re
from collections import defaultdict

createdata = False

src = "cocodoomData/"
width = 320
height = 200

names = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 106, 108, 109, 110, 117, 118, 119, 121, 123, 124, 125, 126, 127, 134]

for fname in ["run-train.json","run-val.json", "run-test.json"]:
    f = json.load(open("cocodoomData/"+fname,"r"))
    idImagesLink = {}
    for images in f["images"]:
        idImagesLink[images["id"]] = {"file_name":images["file_name"], "bbox":[], "category_id":[]}

    for annotation in f["annotations"]:
        idImagesLink[annotation["image_id"]]["bbox"].append(annotation["bbox"])
        idImagesLink[annotation["image_id"]]["category_id"].append(annotation["category_id"])
    count = defaultdict(int)
    for elem in idImagesLink.values():
        for cat in elem["category_id"]:
            count[cat] += 1

    print(sorted(count.items(), key=lambda x: x[1], reverse=True))

    if (createdata):
        for elem in idImagesLink:              
            dest = "data/"+idImagesLink[elem]["file_name"].replace("/rgb","images")
            dest = re.sub(r'\bmap\d{1,2}', '', dest)
            dest = dest.split('/')
            dest[1], dest[2] = dest[2], dest[1]
            dest = '/'.join(dest)
            print(dest)
            shutil.copy(src+idImagesLink[elem]["file_name"], dest)
            with open(dest.replace("images","labels").replace(".png",".txt"),"w") as f:
                # if len(idImagesLink[elem]["bbox"]) == 0:
                #     f.write(str(0)+"\n")
                for i, bbox in enumerate(idImagesLink[elem]["bbox"]):
                    xmin = bbox[0]
                    ymin = bbox[1]
                    xmax = bbox[0]+bbox[2]
                    ymax = bbox[1]+bbox[3]
                    center_x = min(((xmin+xmax)//2)/width,1.0)
                    center_y = min(((ymin+ymax)//2)/height,1.0)
                    widthbb = min((xmax-xmin)/width,1)
                    heightbb = min((ymax-ymin)/height,1)
                    f.write(str(names.index(idImagesLink[elem]["category_id"][i]))+" "+str(center_x)+" "+str(center_y)+" "+str(widthbb)+" "+str(heightbb)+"\n")
    



[(11, 38971), (38, 13403), (12, 12770), (14, 11754), (2, 11524), (18, 7090), (10, 6893), (1, 6760), (37, 5912), (77, 5338), (34, 5212), (45, 4764), (54, 4686), (17, 4243), (20, 4212), (8, 3898), (33, 3870), (73, 3768), (5, 3738), (53, 3549), (93, 2865), (46, 2666), (23, 2639), (30, 2543), (63, 2424), (42, 2272), (31, 2262), (92, 1932), (66, 1764), (91, 1728), (22, 1708), (64, 1637), (70, 1476), (15, 1464), (32, 1374), (65, 1350), (69, 1302), (39, 1140), (35, 1082), (67, 970), (106, 887), (9, 801), (59, 723), (3, 721), (82, 656), (95, 650), (57, 650), (126, 594), (97, 593), (55, 586), (19, 548), (68, 541), (36, 541), (6, 535), (127, 534), (75, 469), (16, 450), (43, 448), (21, 443), (96, 410), (90, 388), (71, 369), (121, 367), (83, 366), (56, 339), (76, 315), (89, 300), (62, 297), (124, 292), (58, 286), (7, 283), (78, 273), (100, 265), (109, 263), (88, 240), (72, 197), (94, 178), (99, 177), (110, 176), (44, 175), (118, 155), (108, 151), (84, 141), (74, 140), (80, 140), (60, 129), (134, 1

On peut observer que le dataset est fortment désequilibrer ce qui peut dégrader les resultat de notre modèle, cependant rééquilibgrer le dataset est une tache délicate. Deux option s'offre a nous: l'undersampling et l'oversampling. Dans notre cas, il est compliqué de faire de l'undersampling car les images contenant les classe majoritaire conteinnet sans doute aussi des objets d'autre classe. L'oversampling des classe minoritaire est tout aussi compliqué pour la même raison. Nous avons donc décidé de ne pas rééquilibrer le dataset.

## Config

Nous devons crée un fichier de config pour l'entrainement du model. Ce fichier contient les informations suivantes :
- path: path vers les images d'entrainement
- train: path vers le fichier contenant les images d'entrainement
- valid: path vers le fichier contenant les images de validation
- names: Ids et noms des classes

In [ ]:
import yaml

data = dict(
path= '/Users/engel/Documents/cocoDoom/datatest',
train='images/run1',
val= 'images/run1',

names = 
{
0 :  "a" ,
1 :  "b" ,
2 :  "c" ,
3 :  "d" ,
4 :  "e" ,
5 :  "f" ,
6 :  "g" ,
7 :  "h" ,
8 :  "i" ,
9 :  "j" ,
10 :  "k" ,
11 :  "l" ,
12 :  "m" ,
13 :  "o" ,
14 :  "p" ,
15 :  "q" ,
16 :  "r" ,
17 :  "s" ,
18 :  "t" ,
19 :  "u" ,
20 :  "v" ,
21 :  "w" ,
22 :  "x" ,
23 :  "z" ,
24 :  "A" ,
25 :  "B" ,
26 :  "C" ,
27 :  "D" ,
28 :  "E" ,
29 :  "F" ,
30 :  "G" ,
31 :  "H" ,
32 :  "I" ,
33 :  "J" ,
34 :  "K" ,
35 :  "L" ,
36 :  "M" ,
37 :  "O" ,
38 :  "P" ,
39 :  "Q" ,
40 :  "R" ,
41 :  "S" ,
42 :  "T" ,
43 :  "U" ,
44 :  "V" ,
45 :  "W" ,
46 :  "X" ,
47 :  "Z" ,
48 :  "aa" ,
49 :  "ba" ,
50 :  "ca" ,
51 :  "da" ,
52 :  "ea" ,
53 :  "fa" ,
54 :  "ga" ,
55 :  "ha" ,
56 :  "ia" ,
57 :  "ja" ,
58 :  "ka" ,
59 :  "la" ,
60 :  "ma" ,
61 :  "oa" ,
62 :  "pa" ,
63 :  "qa" ,
64 :  "ra" ,
65 :  "sa" ,
66 :  "ta" ,
67 :  "ua" ,
68 :  "va" ,
69 :  "wa" ,
70 :  "xa" ,
71 :  "za" ,
72 :  "Aa" ,
73 :  "Ba" ,
74 :  "Ca" ,
75 :  "Da" ,
76 :  "Ea" ,
77 :  "Fa" ,
78 :  "Ga" ,
79 :  "Ha" ,
80 :  "Ia" ,
81 :  "Ja" ,
82 :  "Ka" ,
83 :  "La" ,
84 :  "Ma" ,
85 :  "Oa" ,
86 :  "Pa" ,
87 :  "Qa" ,
88 :  "Ra" ,
89 :  "Sa" ,
90 :  "Ta" ,
91 :  "Ua" ,
92 :  "Va" ,
93 :  "Wa" ,

}
)

with open('data2.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


## Entrainement du modèle
Nous pouvons désormais entrainer notre modèle sur notre dataset, nous utilison un modèle préetnainé sur le dataset 'COCO' pour la detection/segmentation et sur el dataset 'Imagenet' pour la classification. Nous avons entrainé le modèle sur 15 epochs car au delà on depasse les douzes heures maximus de run de kaggle.

In [ ]:
!wandb disabled
!yolo detect train data='config.yaml' model='yolov8n.pt' epochs=15

## Prédictions
prediction sur des nouvelle donnée qu'on dump en json 

In [ ]:
from ultralytics import YOLO


model = YOLO('results/runs/detect/train/weights/best.pt')  # initialize
result = model.predict('data/images/run3/', conf=0.75)  # predict

r = {}
for i in result:
    r[i.path] = (i.tojson())
json.dump(r, open("resultconf0.75.json","w"))

## Evaluation
Nous pouvons désormais évaluer les performances de notre modèle. Pour ce faire nous devons charger les données prédites ainsi que les données réeles afin de pouvoir les comparer.

In [ ]:
Data = json.load(open("resultconf0.75.json","r"))
predictedData = {}
for key, elem in Data.items():
    print(key, elem)
    predictedData[key] = json.loads(elem)

f = json.load(open("cocodoomData/run-test.json","r"))
groundTruthData = {}
for images in f["images"]:
    groundTruthData[images["id"]] = {"file_name":images["file_name"], "bbox":[], "category_id":[]}

for annotation in f["annotations"]:
    box = annotation["bbox"]
    box[2] += box[0]
    box[3] += box[1]
    groundTruthData[annotation["image_id"]]["bbox"].append(box)
    groundTruthData[annotation["image_id"]]["category_id"].append(annotation["category_id"])

In [ ]:
def computeIoU(predictedBox, trueBox):
    x1 = max(predictedBox[0], trueBox[0])
    y1 = max(predictedBox[1], trueBox[1])
    x2 = min(predictedBox[2], trueBox[2])
    y2 = min(predictedBox[3], trueBox[3])
    intersection = max(0, x2-x1) * max(0, y2-y1)
    union =( predictedBox[2]-predictedBox[0])*(predictedBox[3]-predictedBox[1]) + (trueBox[2]-trueBox[0])*(trueBox[3]-trueBox[1]) - intersection
    # print("Intersection: ", intersection)
    # print("Union: ", union)
    return intersection/union

Une fois les données chargées, nous pouvons calculer les metrics suivants:
...
...
...

calcul des metrics, je vais probablment tout changer demain il y a des trucs préfait par yolo

In [ ]:
iou = {}
iouMoyenne = 0
falsePrediction = 0
totErrorNbBox = 0
total = 0
for predicetedkey, predictedvalue,truevalue in zip(predictedData.keys(), predictedData.values(),groundTruthData.values()):
    iou[predicetedkey] = {'metric':[]}
    ErrorNbBox = 0
    if (len(predictedvalue) != len(truevalue["bbox"])):
        if (len(predictedvalue) > len(truevalue["bbox"])):
            ErrorNbBox = len(predictedvalue) - len(truevalue["bbox"])
        else:
            ErrorNbBox = len(truevalue["bbox"]) - len(predictedvalue)
        totErrorNbBox += abs(ErrorNbBox)
    for trueBox,trueid in zip(truevalue["bbox"], truevalue["category_id"]):
        computedIoU = 0
        predictedIdMaxIoU = 0
        for predicted in predictedvalue:
            predictedBox = [predicted['box']["x1"], predicted['box']["y1"], predicted['box']["x2"], predicted['box']["y2"]]
            predictedId = predicted["class"]+1
            newIoU = computeIoU(predictedBox, trueBox)
            if (computedIoU<newIoU):
                computedIoU = newIoU
                predictedIdMaxIoU = predictedId
        if (len(predictedvalue) != 0):
            falsePrediction += 1 if predictedIdMaxIoU!=trueid else 0
            if predictedIdMaxIoU!=trueid:
                print("Predicted ID: ", predictedId)
                print("True ID: ", trueid)
        total += 1
        iouMoyenne += computedIoU
        iou[predicetedkey]['metric'].append({ "iou": computedIoU, "predictedId": predictedId})
        iou[predicetedkey]['errorNbBox'] = ErrorNbBox
            



print("IoU Moyenne: ", iouMoyenne/total)
print("False Prediction: ", falsePrediction)
print("Total: ", total)
print("Total Error Nb Box: ", totErrorNbBox)
iou['globalMetrics'] = {"IoUMoyenne": iouMoyenne/total, "FalsePrediction": falsePrediction, "Total": total, "TotalErrorNbBox": totErrorNbBox}
json.dump(iou, open("iouconf0.75.json","w"))


In [ ]:
for modelstr in ['resultLarge30', 'resultLarge', 'results']:  #dans l'ordre c'est val3, val4 et val 5
    model = YOLO(f'{modelstr}/runs/detect/train/weights/best.pt')
    res = model.val()

# Amélioration du modele

Les même manipulation on été faites pour 2 autres modèles: le modèleyoloV8l entrainé sur 15 epochs et le modèles yolov8l entrainé sur 30 epochs (en réentrainant le modele a partir des poids de l'entrainement 15 epochs)

Le modèle yolov8l (large) est un plus gros que yolov8n (nano). Ce qui le rends plus lent mais plus précis. (cfr https://docs.ultralytics.com/tasks/detect/)

# Resultat et interpretation


<img src="resultLarge30\runs\detect\train\val_batch1_labels.jpg" alt= “” width="value" height=300>


<img src="resultLarge30\runs\detect\train\val_batch1_pred.jpg" alt= “” width="value" height=300>

## confusions matrix

EN analisant la matrice de confusion resultant de l'entrainement du modele on peut observer que l'erreur la plus prédominante est la detection d'un objet comme etant un background. Cela veut dire que le modèle ne détecte rien dans l'image, cette erreur peut etre due a un manque d'entrainement ou a un threshold de confiance trop élevé. L'erreur inverse apparait aussi, c'est a dire que le modèle detecte un background comme etant un objet, cela est sans doute aussi du a un manque d'entrainement ou a un threshold de confiance trop bas. Plusieur classe sont aussi regulierement confonudes, probablment du a une ressemblance entre les objets de ces classes.

Les matrices sont similaire pour les 3 modèles.

<img src="resultLarge30\runs\detect\train\confusion_matrix_normalized.png" alt= “” width="value" height=600>


En analysant les courbe fournie par yolo apres l'enrtainement on peu tobserver que la loss diminue au fur et a mesure des epochs, ce qui est normal. Cependant elle n'atteind j'amais de plateau, ce qui inquie que le modèle pourrait encore s'améliorer si on l'entrainait plus longtemps.

En comparant les donnée, on peut voir que le modele entrainé sur 30 epochs a une loss de validation plus faible. De plus les metrics map, précision et recall de celui-ci sont aussi plus elevée.(mAP0.5 represente la mean average precission avec un seuil de IoU de 0.5 pour determiner le true positives ).

Il y a peu de différence entre les deux modèles entrainés sur 15 epochs, cependant il est probable qu'avec un entrainement plus long, le modèle yolov8l (large) surpasse le modèle yolov8n (nano).

Le model entrainé sur 30 epochs est donc le meilleur des 3.



large30
<img src="resultLarge30\runs\detect\train\results.png" alt= “” width="value" height=600>

large
<img src="resultLarge\runs\detect\train\results.png" alt= “” width="value" height=600>

nano
<img src="results\runs\detect\train\results.png" alt= “” width="value" height=600>

Une autre metric interessante est le Iou (intersection over Union) qui compare les bounding box prédite au bounding box réel.

pour nos trois models nous avons un IoU moyen de:

| Seuil de confiance/model | yolovv8n | yolov8L 15 epochs | yolov8L 30 epochs |
|---|---|---|---|
| 0.25 | 0.671 | 0.671 | 0.682 |
| 0.5 | 0.593 | 0.593 | 0.606 |
| 0.75 | 0.450 | 0.450 | 0.459 |

a


| Seuil de confiance/model | yolovv8n | yolov8L 15 epochs | yolov8L 30 epochs |
|---|---|---|---|
| 0.25 | +: 1507 -: 3946 | +: 1507 -: 3946 | +: 1589 -: 3780 |
| 0.5 | +: 126 -: 8363 | +: 126 -: 8363 | +: 106 -: 8065 |
| 0.75 | +: 11 -: 13100 | +: 11 -: 13100 | +: 14 -: 12901 |

De plus, l'entrainement du modèle nous fourni 4 courbe qui permettent d'annalysé les performances du modèle en fonction de seuil de confiance afin de fine-tuner le modèle selon nos besoins.

<img src="resultLarge30\runs\detect\train\F1_curve.png" alt= “” width="value" height=600>
<img src="resultLarge30\runs\detect\train\P_curve.png" alt= “” width="value" height=600>
<img src="resultLarge30\runs\detect\train\R_curve.png" alt= “” width="value" height=600>
<img src="resultLarge30\runs\detect\train\PR_curve.png" alt= “” width="value" height=600>

## Conclusion
Nous avons vu que l'un des facteurs limitant la performance de notre modèle est le nombre d'epochs d'entrainement. En effet, nous avons entrainé notre modèle sur 15 epochs car au delà on depasse les douzes heures maximus de run de kaggle. Une manière de contourner ce modèle et de réentrainer notre modèle sur 15 epochs en utilisant les poids du modèle précédent. On pourrais ainsi continuer à entrainer notre modèle sur 15 epochs jusqu'à ce que les performances ne s'améliorent plus.
Un autre facteur important est la taille du modèle. En effet, nous avons commencé avec yolo nano avant de passer a yolo large. Nous avons pu constater que le modèle large est plus performant que le modèle nano. Cependant, le modèle large est plus long à entrainer et plus long à prédire. Il serait donc intéressant de trouver un compromis entre la taille du modèle et ses performances.


amelioration :
- mettre vrai nom de classe
- entrainer plus longtemps
- entrainer sur plus de données (utiliser les donnée test pour entrainer)
- fine tune le seuil de confiance